In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
HOME_PATH = '' #home path of the project
TRAIN_FILE = 'REAL DATASETS/TRAIN DATASETS/E_PimaIndiansDiabetes_Real_Train.csv'
SYNTHETIC_FILE = 'SYNTHETIC DATASETS/WGANGP/E_PimaIndiansDiabetes_Synthetic_WGANGP.csv'

In [3]:
#define directory of functions and actual directory
FUNCTIONS_DIR = HOME_PATH + 'STDG APPROACHES'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(FUNCTIONS_DIR)

#import functions for univariate resemblance analisys
from preprocessing import DataPreProcessor

#change directory to actual directory
os.chdir(ACTUAL_DIR)

from ydata_synthetic.synthesizers.regular import WGAN_GP
print('Functions imported!!')

Functions imported!!


## Data Preprocessing

In [4]:
import pandas as pd
real_data = pd.read_csv(HOME_PATH + TRAIN_FILE)
real_data['Outcome'] = real_data['Outcome'].astype('category')
data_cols = real_data.columns
data_train = real_data
real_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,84,0,0,0,0.0,0.304,21,0
1,9,112,82,24,0,28.2,1.282,50,1
2,1,139,46,19,83,28.7,0.654,22,0
3,0,161,50,0,0,21.9,0.254,65,0
4,6,134,80,37,370,46.2,0.238,46,1
...,...,...,...,...,...,...,...,...,...
609,5,139,64,35,140,28.6,0.411,26,0
610,1,96,122,0,0,22.4,0.207,27,0
611,10,101,86,37,0,45.6,1.136,38,1
612,0,141,0,0,0,42.4,0.205,29,1


In [5]:
# data configuration
preprocessor = DataPreProcessor(data_train)
data_train = preprocessor.preprocess_train_data()
data_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome0,Outcome1
0,-0.526397,-1.151398,-3.752683,-1.322774,-0.701206,-4.135256,-0.490735,-1.035940,0.728816,-0.728816
1,1.588046,-0.276643,0.680345,0.233505,-0.701206,-0.489169,2.415030,1.487101,-1.372089,1.372089
2,-0.828460,0.566871,-1.265862,-0.090720,0.013448,-0.424522,0.549161,-0.948939,0.728816,-0.728816
3,-1.130523,1.254179,-1.049617,-1.322774,-0.701206,-1.303720,-0.639291,2.792122,0.728816,-0.728816
4,0.681856,0.410665,0.572222,1.076490,2.484601,1.838121,-0.686829,1.139095,-1.372089,1.372089
...,...,...,...,...,...,...,...,...,...,...
609,0.379793,0.566871,-0.292759,0.946800,0.504235,-0.437451,-0.172824,-0.600933,0.728816,-0.728816
610,-0.828460,-0.776503,2.842797,-1.322774,-0.701206,-1.239073,-0.778934,-0.513932,0.728816,-0.728816
611,1.890109,-0.620297,0.896590,1.076490,-0.701206,1.760544,1.981245,0.443084,-1.372089,1.372089
612,-1.130523,0.629354,-3.752683,-1.322774,-0.701206,1.346804,-0.784877,-0.339929,-1.372089,1.372089


## Train the Model

Next, lets define the neural network for generating synthetic data. We will be using a [GAN](https://www.wikiwand.com/en/Generative_adversarial_network) network that comprises of an generator and discriminator that tries to beat each other and in the process learns the vector embedding for the data. 

The model was taken from a [Github repository](https://github.com/ydataai/gan-playground) where it is used to generate synthetic data on credit card fraud data. 

In [6]:
# training configuration
noise_dim = 32
dim = 128
batch_size = 16

log_step = 200
epochs = 1000+1
learning_rate = 5e-4

beta_1 = 0.5
beta_2 = 0.9

models_dir = 'my_model_datasetE/'

data_dim = data_train.shape[1]
print('Shape of data: ', data_train.shape)

Shape of data:  (614, 10)


In [7]:
#Define the GAN and training parameters
gan_args = [batch_size, learning_rate, beta_1, beta_2, noise_dim, data_dim, dim]
train_args = ['', epochs, log_step]

Finally, let's run the training and see if the model is able to learn something.

In [8]:
!mkdir my_model_datasetE
!mkdir my_model_datasetE/gan
!mkdir my_model_datasetE/gan/saved

In [9]:
#Training the GAN model chosen: Vanilla GAN, CGAN, DCGAN, etc.
synthesizer = WGAN_GP(gan_args, n_critic=2)
synthesizer.train(data_train, train_args)

  1%|▏         | 13/1001 [00:01<01:43,  9.52it/s]

Iteration: 0 | disc_loss: 2.2206082344055176 | gen_loss: 0.046485912054777145
Iteration: 1 | disc_loss: 0.475229948759079 | gen_loss: 0.009209999814629555
Iteration: 2 | disc_loss: 0.22922585904598236 | gen_loss: -0.03548869490623474
Iteration: 3 | disc_loss: 0.2156958431005478 | gen_loss: -0.09174392372369766
Iteration: 4 | disc_loss: 0.28119486570358276 | gen_loss: -0.13260185718536377
Iteration: 5 | disc_loss: 0.15668651461601257 | gen_loss: -0.2306608408689499
Iteration: 6 | disc_loss: 0.09555962681770325 | gen_loss: -0.2177928388118744
Iteration: 7 | disc_loss: 0.2926592230796814 | gen_loss: -0.2336753010749817
Iteration: 8 | disc_loss: -0.0315244197845459 | gen_loss: -0.1974164843559265
Iteration: 9 | disc_loss: -0.1102328673005104 | gen_loss: -0.09878423810005188
Iteration: 10 | disc_loss: -0.35564684867858887 | gen_loss: 0.005152210593223572
Iteration: 11 | disc_loss: -0.5431320667266846 | gen_loss: 0.15287786722183228
Iteration: 12 | disc_loss: -0.7085065245628357 | gen_loss: 

  4%|▍         | 40/1001 [00:02<00:27, 34.87it/s]

Iteration: 23 | disc_loss: -0.6412625908851624 | gen_loss: 0.004161694552749395
Iteration: 24 | disc_loss: -0.6370314359664917 | gen_loss: -0.03681158274412155
Iteration: 25 | disc_loss: -0.6920619010925293 | gen_loss: -0.07218147814273834
Iteration: 26 | disc_loss: -0.6053986549377441 | gen_loss: -0.020312249660491943
Iteration: 27 | disc_loss: -0.6757109761238098 | gen_loss: -0.002293115947395563
Iteration: 28 | disc_loss: -0.6281484961509705 | gen_loss: 0.013417450711131096
Iteration: 29 | disc_loss: -0.6575934290885925 | gen_loss: 0.011413928121328354
Iteration: 30 | disc_loss: -0.510212779045105 | gen_loss: 0.02309674769639969
Iteration: 31 | disc_loss: -0.6285193562507629 | gen_loss: 0.0033182771876454353
Iteration: 32 | disc_loss: -0.6243249177932739 | gen_loss: -0.010210791602730751
Iteration: 33 | disc_loss: -0.6638768911361694 | gen_loss: -0.0327579639852047
Iteration: 34 | disc_loss: -0.6276376843452454 | gen_loss: -0.02428426966071129
Iteration: 35 | disc_loss: -0.546649396

  7%|▋         | 67/1001 [00:02<00:14, 63.62it/s]

...

Iteration: 960 | disc_loss: -0.27184122800827026 | gen_loss: 0.09282097965478897
Iteration: 961 | disc_loss: -0.34138405323028564 | gen_loss: 0.08529850840568542
Iteration: 962 | disc_loss: -0.2964473068714142 | gen_loss: 0.04710981249809265
Iteration: 963 | disc_loss: -0.22613893449306488 | gen_loss: -0.02447466552257538
Iteration: 964 | disc_loss: -0.4243786334991455 | gen_loss: 0.08287028223276138
Iteration: 965 | disc_loss: -0.34675806760787964 | gen_loss: 0.06562072038650513
Iteration: 966 | disc_loss: -0.43101152777671814 | gen_loss: 0.09929778426885605
Iteration: 967 | disc_loss: -0.28246814012527466 | gen_loss: 0.09286718815565109
Iteration: 968 | disc_loss: -0.1786583662033081 | gen_loss: 0.056170009076595306
Iteration: 969 | disc_loss: -0.2723776400089264 | gen_loss: 0.01156188640743494
Iteration: 970 | disc_loss: -0.12920312583446503 | gen_loss: -0.04106516391038895
Iteration: 971 | disc_loss: -0.167951762676239 | gen_loss: 0.10663839429616928
Iteration: 972 | disc_loss: -0.

100%|█████████▉| 997/1001 [00:10<00:00, 97.67it/s] 

Iteration: 980 | disc_loss: -0.34265846014022827 | gen_loss: 0.045369379222393036
Iteration: 981 | disc_loss: -0.37006548047065735 | gen_loss: 0.016419317573308945
Iteration: 982 | disc_loss: -0.15122586488723755 | gen_loss: 0.002808661200106144
Iteration: 983 | disc_loss: -0.2652285099029541 | gen_loss: -0.05012625455856323
Iteration: 984 | disc_loss: -0.33038508892059326 | gen_loss: 0.024197692051529884
Iteration: 985 | disc_loss: -0.32812249660491943 | gen_loss: 0.03895813971757889
Iteration: 986 | disc_loss: -0.2968984842300415 | gen_loss: -0.009329421445727348
Iteration: 987 | disc_loss: -0.2702362537384033 | gen_loss: -0.043992847204208374
Iteration: 988 | disc_loss: -0.33819374442100525 | gen_loss: -0.05743469297885895
Iteration: 989 | disc_loss: -0.3074600398540497 | gen_loss: 0.040840309113264084
Iteration: 990 | disc_loss: -0.26436150074005127 | gen_loss: -0.08588804304599762
Iteration: 991 | disc_loss: -0.29548466205596924 | gen_loss: 0.07749314606189728
Iteration: 992 | dis

100%|██████████| 1001/1001 [00:10<00:00, 95.53it/s]

Iteration: 1000 | disc_loss: -0.21665635704994202 | gen_loss: 0.12537842988967896


In [10]:
synthesizer.generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(16, 32)]                0         
_________________________________________________________________
dense (Dense)                (16, 128)                 4224      
_________________________________________________________________
dense_1 (Dense)              (16, 256)                 33024     
_________________________________________________________________
dense_2 (Dense)              (16, 512)                 131584    
_________________________________________________________________
dense_3 (Dense)              (16, 10)                  5130      
Total params: 173,962
Trainable params: 173,962
Non-trainable params: 0
_________________________________________________________________


In [11]:
synthesizer.critic.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(16, 10)]                0         
_________________________________________________________________
dense_4 (Dense)              (16, 512)                 5632      
_________________________________________________________________
dropout (Dropout)            (16, 512)                 0         
_________________________________________________________________
dense_5 (Dense)              (16, 256)                 131328    
_________________________________________________________________
dropout_1 (Dropout)          (16, 256)                 0         
_________________________________________________________________
dense_6 (Dense)              (16, 128)                 32896     
_________________________________________________________________
dense_7 (Dense)              (16, 1)                   129 

## Generate data

In [12]:
size = len(data_train)
generated_samples = synthesizer.sample(size)
generated_samples.columns = data_train.columns
generated_samples

Synthetic data generation: 100%|██████████| 39/39 [00:00<00:00, 156.88it/s]


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome0,Outcome1
0,-0.304917,0.247180,0.420425,-0.820902,-0.479163,-0.576614,-0.410530,0.390056,0.428197,-0.457444
1,-0.424372,0.195200,0.370187,-0.640852,-0.328605,-0.711391,-0.360576,0.077787,0.374991,-0.385800
2,-0.424870,0.223621,0.560748,-0.824561,-0.406687,-0.650231,-0.468318,0.490581,0.544008,-0.533039
3,-0.450998,0.286754,0.500671,-0.780538,-0.405342,-0.851185,-0.385848,0.245606,0.541447,-0.491871
4,-0.604380,-0.176657,0.502668,-0.596138,-0.287006,-0.564463,-0.452728,0.005031,0.549342,-0.537214
...,...,...,...,...,...,...,...,...,...,...
11,-0.236351,0.230217,0.394168,-0.734280,-0.431312,-0.464181,-0.390306,0.524391,0.433226,-0.433444
12,-0.450421,0.381196,-1.901278,-0.852155,-0.414534,-0.596910,-0.611733,-0.271174,-0.460098,0.464061
13,-0.387946,0.117355,0.406001,-0.841617,-0.413243,-0.452135,-0.388038,0.091337,0.323719,-0.297584
14,-0.351638,-0.176593,0.294202,-0.652789,-0.404151,-0.493682,-0.363932,0.186489,0.118121,-0.171466


## Transform and process generated data

In [13]:
synthetic_data = preprocessor.transform_data(generated_samples)
synthetic_data = synthetic_data[0:len(real_data)]
synthetic_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,128,77,7,25,27.523672,0.330995,37,0
1,2,127,76,10,43,26.481262,0.347808,33,0
2,2,128,79,7,34,26.954290,0.311545,38,0
3,2,130,78,8,34,25.400051,0.339302,35,0
4,1,115,78,11,48,27.617651,0.316792,32,0
...,...,...,...,...,...,...,...,...,...
1,2,117,71,14,60,28.184755,0.345568,30,0
2,2,123,75,7,30,29.951437,0.359797,35,0
3,2,126,74,11,53,27.091782,0.317798,33,0
4,2,129,74,9,47,26.547909,0.326646,33,0


In [14]:
print(real_data.dtypes, '\n', synthetic_data.dtypes)
print(real_data.shape, synthetic_data.shape)

Pregnancies                    int64
Glucose                        int64
BloodPressure                  int64
SkinThickness                  int64
Insulin                        int64
BMI                          float64
DiabetesPedigreeFunction     float64
Age                            int64
Outcome                     category
dtype: object 
 Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object
(614, 9) (614, 9)


In [15]:
real_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.742671,120.855049,69.415309,20.399023,81.438111,31.983388,0.469168,32.907166
std,3.313264,32.035057,18.512599,15.433974,116.234835,7.740625,0.336847,11.503437
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,100.000000,64.000000,0.000000,0.000000,27.100000,0.241500,24.000000
50%,3.000000,117.000000,72.000000,23.000000,42.500000,32.000000,0.372500,29.000000
75%,6.000000,139.000000,80.000000,32.000000,129.750000,36.375000,0.613750,40.000000
max,17.000000,199.000000,122.000000,63.000000,846.000000,67.100000,2.420000,81.000000


In [16]:
synthetic_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,1.929967,122.359935,71.265472,9.376221,41.030945,27.579839,0.330722,33.040717,0.086319
std,0.559638,5.849033,9.158539,2.717262,15.257572,1.352330,0.033380,2.917568,0.281064
min,0.000000,107.000000,19.000000,3.000000,3.000000,23.501244,0.218074,24.000000,0.000000
25%,2.000000,119.000000,70.000000,7.000000,30.000000,26.694817,0.309488,31.000000,0.000000
50%,2.000000,123.000000,74.000000,9.000000,39.000000,27.546986,0.328180,33.000000,0.000000
75%,2.000000,126.000000,77.000000,11.000000,52.000000,28.494532,0.349515,35.000000,0.000000
max,5.000000,141.000000,82.000000,18.000000,84.000000,31.063000,0.509714,40.000000,1.000000


In [17]:
synthetic_data.groupby('Outcome').count()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,561,561,561,561,561,561,561,561
1,53,53,53,53,53,53,53,53


In [18]:
#Save generated samples
synthetic_data.to_csv(HOME_PATH + SYNTHETIC_FILE, index=False)